### SJHR3D Model with Stimulus

In [1]:
# load library
import numpy as np
import scipy
import os
import json
import matplotlib.pyplot as plt
from tvb.simulator.lab import *
from tvb.basic.readers import FileReader

In [ ]:
# normalized weight
mat = scipy.io.loadmat('...SC.mat')
SC = mat['SC_cap_agg_bwflav2']
SCnorm = np.log(SC+1)/np.log(SC+1).sum(axis=0).max()

matfile = '...weigth.txt'
np.savetxt(matfile, SCnorm)

In [3]:
# path to files

# insert the path to where all subjects connectivities are stored
mypath = ""
# specify which subject to load (folder name of each subject)
subject = ""
# save path
save_path = ""

# load files

# load connectivity weights
reader = FileReader(file_path= mypath +'/' + subject +'/weights.txt')
w= reader.read_array(dtype=np.float64, skip_rows=0, use_cols=None, matlab_data_name=None)

# load region centers
reader = FileReader(file_path=mypath +'/' + subject +'/centres.txt')    
rl= reader.read_array(dtype=np.str_, skip_rows=0, use_cols=(0,), matlab_data_name=None)
c= reader.read_array(dtype=np.float64, skip_rows=0, use_cols=(1, 2, 3), matlab_data_name=None)

# load connectivity tract lengths
reader = FileReader(file_path=mypath +'/' + subject +'/tract_lengths.txt')
tl= reader.read_array(dtype=np.float64, skip_rows=0, use_cols=None, matlab_data_name=None)

In [6]:
# config model

# csf (global coupling)
csf = np.array([0.13])
# conduction speed 
speed = 40

# config the connectivity
white_matter = connectivity.Connectivity(region_labels=rl, weights=w, tract_lengths=tl, centres=c)
white_matter.configure()

# setup the simulation
oscilator = models.ReducedSetHindmarshRose(r=np.array([0.006]), a=np.array([1.0]), b=np.array([3.0]), c=np.array([1.0]), d=np.array([5.0]), 
                                            s=np.array([4.0]), xo=np.array([-1.6]), 
                                            K11=np.array([0.5]), K12=np.array([0.1]), K21=np.array([0.15]), 
                                            sigma=np.array([0.3]), mu=np.array([2.2]), variables_of_interest=["xi","alpha"])
oscilator.configure()

dt=0.01220703125

# specify the coupling function
white_matter_coupling = coupling.Linear(a=csf, b=np.array([0.0]))
white_matter_coupling.configure()

# set up the integration
heunint = integrators.HeunStochastic(dt=dt, noise=noise.Additive(nsig=np.array([1.0])))

# specify tvb monitors
p=3.90625 # 256Hz = 3.90625 ms
momo = monitors.SubSample(period=p) # collect data at every 3.90635 ms

# choose to generate BOLD signal
hrffunction=equations.MixtureOfGammas()
pb=500 
mama = monitors.Bold(period=pb, hrf_kernel=hrffunction)

# put both monitors together
what_to_watch = (momo, mama)
# time in sec
sim_length = 180000

### Stimulus

In [10]:
atlas = 'Desikan-Killiany'
SimNIBSfile = '...efield.json'

# for reading e-field in json file
with open(SimNIBSfile, 'r') as f:
    EF_data = json.load(f)

# get region labels and index
regions_li = white_matter.get_grouped_space_labels()

# edit brain area name
ef_idxRegions = EF_data.copy()
del ef_idxRegions['lh.corpuscallosum']
del ef_idxRegions['rh.corpuscallosum']

ef_idxRegions = { 'lh_' + k[3:] if 'lh.' in k else 'rh_' + k[3:]: v for k, v in ef_idxRegions.items() }

# match region to tvb index
efield = {}
for key, value in ef_idxRegions.items():
    for hemi, regions in regions_li:
        for idx, area in regions:
            if area.lower() == key.lower():
                efield[(idx, area)] = value

In [15]:
# define the location of stimuli
# use all EF result from all regions
chosenNodes = [i for i, e in efield.keys()]
# Make a matrix of stimuli weights
stim_weights_raw = np.zeros(white_matter.number_of_regions)
chosenNodes_weights = np.array(list(efield.values()))

# normalized e-fields to [-1, 1]
normalized_weights = 2*(chosenNodes_weights - chosenNodes_weights.min()) / (chosenNodes_weights.max() - chosenNodes_weights.min()) - 1
stim_weights_raw[chosenNodes] = normalized_weights

# Quasi-uniform assumption
constant = 4
stim_weights = stim_weights_raw * constant

# so-tDCS 
stim_start = 60000
stim_end = 120000
params = {"amp": 0.5, "frequency": 0.00075, "start": stim_start, "end": stim_end}
equation='where((var > start) & (var < end), amp + amp*sin(6.283185307179586*frequency*var), 0)'
eqn_t = equations.TemporalApplicableEquation(equation=equation, parameters=params)

# configure stimulus
stimulus = patterns.StimuliRegion(
    temporal = eqn_t,
    connectivity = white_matter,
    weight = stim_weights)
stimulus.configure_space()
stimulus.configure_time(np.arange(0, 180000.0, dt)) # dt = time step

In [ ]:
# configure the simulation
sim = simulator.Simulator(model=oscilator, connectivity=white_matter,
                          coupling=white_matter_coupling, conduction_speed=speed,
                          integrator=heunint, monitors=what_to_watch, simulation_length=sim_length,
                          stimulus=stimulus)

sim.configure()

In [19]:
%%capture sim_runtime
%%timeit -n 1 -r 1
# perform the simulation
subs_data = []
subs_time = []
bold_data = []
bold_time = []

for subs, bold in sim():
    if not subs is None:
        subs_time.append(subs[0])
        subs_data.append(subs[1])
    
    if not bold is None:
        bold_time.append(bold[0])
        bold_data.append(bold[1])

SUBS  = np.sum(np.array(subs_data),axis=3) 
TSUBS = np.array(subs_time)
BOLD  = np.array(bold_data)
TBOLD = np.array(bold_time)
subsfile = save_path + '/' + subject + '.mat'
scipy.io.savemat(subsfile, mdict={ 'subs_data': SUBS, 'subs_time': TSUBS ,'bold_data': BOLD, 'bold_time': TBOLD })